In [1]:
from netCDF4 import Dataset, num2date, chartostring
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle 
import numpy as np
import pandas as pd
from numpy import ma
from numba import njit
from numpy.random import lognormal, randint
from scipy import ndimage
from scipy.stats import gamma, rankdata
from scipy.optimize import minimize, minimize_scalar
import math
import warnings
import os
import pickle
import re
from scipy.interpolate import RectBivariateSpline



import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature

import matplotlib.colors as colors

exec(open("/users/jbellier/Documents/Tools/Functions_dates.py").read())
exec(open("/users/jbellier/Documents/Tools/Functions_plots.py").read())
exec(open("/users/jbellier/Documents/Tools/Function_constrNMPy.py").read())
exec(open("/users/jbellier/Documents/Tools/Functions_circularData.py").read())
exec(open("/users/jbellier/Documents/GitHub/GSDM/Functions/Various_functions.py").read())



#### Small sample of 10 coarse-scale and fine-scale fields

In [2]:
dates_to_select = range(46,86)
lb_lat, ub_lat, lb_lon, ub_lon = 31.7, 45.5, -124.8, -113.2

## Fine-scale analyses:
ncfile = '/users/jbellier/Documents/Data/NLDAS/NLDAS-California_20100101_to_20171231_precipOnly_6h_accumulation.nc'
nc = Dataset(ncfile)
lat_fine = nc.variables['latitude'][:]
lon_fine = nc.variables['longitude'][:]
yyyymmddhh_end6h = nc.variables['yyyymmddhh_valid'][dates_to_select]
time_end6h = nc.variables['time'][dates_to_select]
latRes_fine = np.diff(lat_fine)[0]
lonRes_fine = np.diff(lon_fine)[0]
id_lat_fine = np.logical_and(lat_fine >= lb_lat, lat_fine <= ub_lat)
id_lon_fine = np.logical_and(lon_fine >= lb_lon, lon_fine <= ub_lon)
lat_fine = lat_fine[id_lat_fine]
lon_fine = lon_fine[id_lon_fine]
N_fineFields = nc.variables['apcp_anal'][:][dates_to_select,:,:][:,id_lat_fine,:][:,:,id_lon_fine]
nc.close()

N, nya, nxa = N_fineFields.shape

## Load the mask from the California project:
ncfile = '/users/jbellier/Documents/Project_California/downscaled_fields/1stage/E40oS20/downscaled_fields_201610_000_to_003.nc'
nc = Dataset(ncfile)
mask_fine = nc.variables['S1_fields'][0,0,:,:].mask
mask_fine = ma.array(np.zeros((nya,nxa)), mask=True)
pos_lat = fun_matchIndex(lat_fine, nc.variables['latitude_fine'][:])
pos_lon = fun_matchIndex(lon_fine, nc.variables['longitude_fine'][:])
mask_fine[pos_lat[pos_lat.mask==False,None],
          pos_lon[None,pos_lon.mask==False]] = nc.variables['S1_fields'][0,0,:,:].mask[pos_lat.mask==False,:][:,pos_lon.mask==False]
nc.close()

N_fineFields.mask = mask_fine[None,:,:]


lat_fine = coord_equallySpaced(lat_fine)
lon_fine = coord_equallySpaced(lon_fine)



## Coarse-scale grid:
ncfile = '/users/jbellier/Documents/Data/NLDAS/NLDAS-California_20100101_to_20171231_precipOnly_6h_accum_aggregated_to_GEFS_T254.nc'
nc = Dataset(ncfile)
lat_coarse = coord_equallySpaced(nc.variables['latitude_coarse'][:])
lon_coarse = coord_equallySpaced(nc.variables['longitude_coarse'][:])
latRes_coarse = np.diff(lat_coarse)[0]
lonRes_coarse = np.diff(lon_coarse)[0]
id_lat_coarse = np.logical_and(lat_coarse >= lb_lat, lat_coarse <= ub_lat)
id_lon_coarse = np.logical_and(lon_coarse >= lb_lon, lon_coarse <= ub_lon)
lat_coarse = lat_coarse[id_lat_coarse]
lon_coarse = lon_coarse[id_lon_coarse]
# N_coarseFields = nc.variables['apcp_anal'][:][dates_to_select,:,:]
# assert np.all(yyyymmddhh_end6h == nc.variables['yyyymmddhh_valid'][dates_to_select])
nc.close()

nyc, nxc = lat_coarse.size, lon_coarse.size


#### Predictors:

In [4]:
# lb_lat, ub_lat, lb_lon, ub_lon = np.min(lat_fine), np.max(lat_fine), np.min(lon_fine), np.max(lon_fine)
name_area = 'California_Nevada'

list_namePred = ['WindSpeed','WindDir','WindSpeed','WindDir','MUCAPE','WindShearSpeed','LI']
list_level_or_layer = [600,600,700,700,'None','sfc-700hPa','None']
nb_diffPred = len(list_namePred)

## Load the meteorological predictor original grid (ERA5):
#---------------------------------------------
# WARNING: lats are here in decreasing order!
nc = Dataset('/users/jbellier/Documents/Data/ERA5/files/Orography/Orography.nc')
lons_pred = nc.variables['longitude'][:]
lats_pred = nc.variables['latitude'][:]
res_pred = np.unique(np.diff(lons_pred))[0]
id_lons_pred = np.logical_and(lons_pred >= lb_lon-res_pred, lons_pred <= ub_lon+res_pred)    # These will be used
id_lats_pred = np.logical_and(lats_pred >= lb_lat-res_pred, lats_pred <= ub_lat+res_pred)    # when loading the netcdf
lons_pred = lons_pred[id_lons_pred]
lats_pred = np.flip(lats_pred[id_lats_pred])     # we flip to get back to latitude going in increasing order
nc.close()

nyp, nxp = len(lats_pred), len(lons_pred)

Pred_originRes = ma.array(np.zeros((3,nb_diffPred,N,nyp,nxp), dtype='float64'), mask=True)



## Note: ERA5 is at 6h time step only. But since here we consider TempResol_NLDAS = 6h, we load the predictor data at the beginning and end of the 6h period, and we keep the average of the two.
##   (If TempResol_NLDAS = 3h, we would also take the average of the 6h period, for the two 3h period spanning the 6h period (means that two 3h periods will have the same predictor values).         

## Vector of the beginning of the 6h period:
yyyymmddhh_beg6h = np.zeros(N, dtype='int64')
for n in range(N):
    yyyymmddhh_beg6h[n] = date_to_yyyymmddhh(datetime(yyyy(yyyymmddhh_end6h[n]), mm(yyyymmddhh_end6h[n]), dd(yyyymmddhh_end6h[n]), hh(yyyymmddhh_end6h[n])) - timedelta(hours=6)) 

## yyyymmddhh_beg6h[0] might be in the previous year (occurs in Jan), so we replace by yyyymmddhh_end6h[0]:
if yyyy(yyyymmddhh_beg6h[0]) !=  yyyy(yyyymmddhh_beg6h[1]):
    yyyymmddhh_beg6h[0] = yyyymmddhh_end6h[0]
## Similarly, yyyymmddhh_end6h[-1] might be in the next year (occurs in Dec), so we replace by yyyymmddhh_beg6h[-1]:
if yyyy(yyyymmddhh_end6h[-1]) !=  yyyy(yyyymmddhh_end6h[-2]):
    yyyymmddhh_end6h[-1] = yyyymmddhh_beg6h[-1]


diffyears = np.unique(yyyy(np.append(yyyymmddhh_beg6h, yyyymmddhh_end6h)))



for year in diffyears:

    try:
        ncfile  = '/users/jbellier/Documents/Data/ERA5/Predictors/Predictors_'+str(year)+'_'+name_area+'.nc'
        nc = Dataset(ncfile)

        ## Indices for the instantaneous values at the beginning (B) and end (E) of the 6h periods:
        idB = fun_matchIndex(yyyymmddhh_beg6h, nc.variables['yyyymmddhh_valid'][:])
        idE = fun_matchIndex(yyyymmddhh_end6h, nc.variables['yyyymmddhh_valid'][:])

        idLat = fun_matchIndex(lats_pred, nc.variables['lat'][:])
        idLon = fun_matchIndex(lons_pred, nc.variables['lon'][:])

        for p in range(nb_diffPred):

            namePred = list_namePred[p]
            level_or_layer = list_level_or_layer[p]

            with warnings.catch_warnings():
                warnings.simplefilter('ignore', UserWarning)

                if level_or_layer == 'None':
                    grid_B = nc.variables[namePred] [idB.mask==False,:,:][:,idLat.mask==False,:][:,:,idLon.mask==False]
                    grid_E = nc.variables[namePred] [idE.mask==False,:,:][:,idLat.mask==False,:][:,:,idLon.mask==False]

                else:
                    if nc.variables[namePred].dimensions[1] == 'level':
                        level_or_layer_to_load = np.where(nc.variables['level'][:] == np.int(level_or_layer))[0][0]
                    elif nc.variables[namePred].dimensions[1] == 'layer':
                        level_or_layer_to_load = np.where(chartostring(nc.variables['layer'][:]) == level_or_layer)[0][0]

                    grid_B = nc.variables[namePred] [idB.mask==False,level_or_layer_to_load,:,:][:,idLat.mask==False,:][:,:,idLon.mask==False]
                    grid_E = nc.variables[namePred] [idE.mask==False,level_or_layer_to_load,:,:][:,idLat.mask==False,:][:,:,idLon.mask==False]

            Pred_originRes[0,p,:,:,:][idB[idB.mask==False,None,None],
                                      idLat[None,idLat.mask==False,None],
                                      idLon[None,None,idLon.mask==False]] = grid_B
            Pred_originRes[1,p,:,:,:][idE[idE.mask==False,None,None],
                                      idLat[None,idLat.mask==False,None],
                                      idLon[None,None,idLon.mask==False]] = grid_E    
        nc.close()

    except FileNotFoundError:
        warnings.warn("Tried to load '"+ncfile+"' but file not found.")

## Temporal averaging between the values at the beginning and end of the period:
for p in range(nb_diffPred):
    namePred = list_namePred[p]
    level_or_layer = list_level_or_layer[p]
    if 'Dir' in namePred:    # This predictor is a direction, so we need to perform a circular mean
        Pred_originRes[2,p,:,:,:] = np.rad2deg(circular_mean_element_wise(np.deg2rad(Pred_originRes[0,p,:,:,:]), np.deg2rad(Pred_originRes[1,p,:,:,:])))
    else:                    # "Standard" predictor, so we perform a regular mean
        Pred_originRes[2,p,:,:,:] = 1/2 * (Pred_originRes[0,p,:,:,:] + Pred_originRes[1,p,:,:,:])

        


## Bilinearly interpolated to the coarse-scale:
def iterpdata(lowres,mlats,mlons,lats,lons):
    #  bivariate interpolation
    idata = RectBivariateSpline(mlats,mlons,lowres,kx=1,ky=1,s=0)
    hires = idata.__call__(lats,lons,grid=True)
    return hires

Pred_coarseRes = ma.array(np.zeros((nb_diffPred,N,nyc,nxc), dtype='float64'), mask=True)
for p in range(nb_diffPred):
    for n in range(N):
        Pred_coarseRes[p,n,:,:] = iterpdata(Pred_originRes[2,p,n,:,:], lats_pred,  lons_pred, lat_coarse, lon_coarse)

        


In [10]:
outfilename = '/users/jbellier/Documents/GitHub/GSDM/Data/sample_data.nc'

nc = Dataset(outfilename,'w',format='NETCDF4_CLASSIC')

Dlat_fine_nc = nc.createDimension('lat_fine',nya)
Dlon_fine_nc = nc.createDimension('lon_fine',nxa)
Dlat_coarse_nc = nc.createDimension('lat_coarse',nyc)
Dlon_coarse_nc = nc.createDimension('lon_coarse',nxc)
Ddays = nc.createDimension('time',N)

latitude_fine_nc = nc.createVariable('lat_fine','f8',('lat_fine',))
latitude_fine_nc.long_name = 'latitude fine-scale fields'
latitude_fine_nc.units = 'degrees_north'

longitude_fine_nc = nc.createVariable('lon_fine','f8',('lon_fine',))
longitude_fine_nc.long_name = 'longitude fine-scale fields'
longitude_fine_nc.units = "degrees_east"    

latitude_coarse_nc = nc.createVariable('lat_coarse','f8',('lat_coarse',))
latitude_coarse_nc.long_name = 'latitude coarse-scale fields'
latitude_coarse_nc.units = 'degrees_north'

longitude_coarse_nc = nc.createVariable('lon_coarse','f8',('lon_coarse',))
longitude_coarse_nc.long_name = 'longitude coarse-scale fields'
longitude_coarse_nc.units = "degrees_east"    

yyyymmddhh_end6h_nc = nc.createVariable('yyyymmddhh_end6h','i4',('time',))
yyyymmddhh_end6h_nc.long_name = 'Date/time (yyyymmddhh format, in UTC) of the end of the 6h averaging period'

time_end6h_nc = nc.createVariable('time','i4',('time',))
time_end6h_nc.long_name = 'Date/time (in UTC) of the end of the 6h averaging period'
time_end6h_nc.units = "hours since 1900-01-01 00:00:00"

fine_fields_nc = nc.createVariable('fine_fields','f8',('time','lat_fine','lon_fine',), zlib=True)
fine_fields_nc.long_name = 'Analysis fields'
fine_fields_nc.units = "mm"
fine_fields_nc.valid_range = [0,200]
fine_fields_nc.missing_value = -999

# coarse_fields_nc = nc.createVariable('coarse_fields','f8',('time','lat_coarse','lon_coarse',), zlib=True)
# coarse_fields_nc.long_name = 'Analysis fields aggregated to the coarse-scale'
# coarse_fields_nc.units = "mm"
# coarse_fields_nc.valid_range = [0,200]
# coarse_fields_nc.missing_value = -999

windSpeed_600_nc = nc.createVariable('windSpeed_600','f8',('time','lat_coarse','lon_coarse',), zlib=True)
windSpeed_600_nc.long_name = 'Wind speed (ground-relative) at 600 hPa'
windSpeed_600_nc.units = "m s**1"
windSpeed_600_nc.valid_range = [0,500]
windSpeed_600_nc.missing_value = -9999

windDir_600_nc = nc.createVariable('windDir_600','f8',('time','lat_coarse','lon_coarse',), zlib=True)
windDir_600_nc.long_name = 'Wind direction at 600 hPa. 0 deg means East'
windDir_600_nc.units = "degree"
windDir_600_nc.valid_range = [-180,180]
windDir_600_nc.missing_value = -9999

windSpeed_700_nc = nc.createVariable('windSpeed_700','f8',('time','lat_coarse','lon_coarse',), zlib=True)
windSpeed_700_nc.long_name = 'Wind speed (ground-relative) at 700 hPa'
windSpeed_700_nc.units = "m s**1"
windSpeed_700_nc.valid_range = [0,500]
windSpeed_700_nc.missing_value = -9999

windDir_700_nc = nc.createVariable('windDir_700','f8',('time','lat_coarse','lon_coarse',), zlib=True)
windDir_700_nc.long_name = 'Wind direction at 700 hPa. 0 deg means East'
windDir_700_nc.units = "degree"
windDir_700_nc.valid_range = [-180,180]
windDir_700_nc.missing_value = -9999

CAPE_nc = nc.createVariable('CAPE','f8',('time','lat_coarse','lon_coarse',), zlib=True)
CAPE_nc.long_name = 'Convective Available Potential Energy (most unstable CAPE in the lowest 350hPa, as in ERA5)'
CAPE_nc.units = "J kg**-1"
CAPE_nc.valid_range = [0,99999]
CAPE_nc.missing_value = -9999

WindShearSpeed_nc = nc.createVariable('WindShearSpeed_sfc_700','f8',('time','lat_coarse','lon_coarse',), zlib=True)
WindShearSpeed_nc.long_name = 'Vertical wind shear in speed, between the surface and the 700 hPa level'
WindShearSpeed_nc.units = "m s**11"
WindShearSpeed_nc.valid_range = [0,500]
WindShearSpeed_nc.missing_value = -9999

LI_nc = nc.createVariable('LI','f8',('time','lat_coarse','lon_coarse',), zlib=True)
LI_nc.long_name = 'Lifted Index'
LI_nc.units = "Degree Celsius"
LI_nc.valid_range = [-100,100]
LI_nc.missing_value = -9999

latitude_fine_nc[:] = lat_fine
longitude_fine_nc[:] = lon_fine
latitude_coarse_nc[:] = lat_coarse
longitude_coarse_nc[:] = lon_coarse
yyyymmddhh_end6h_nc[:] = yyyymmddhh_end6h
time_end6h_nc[:] = time_end6h
fine_fields_nc[:,:,:] = N_fineFields
# coarse_fields_nc[:,:,:] = N_coarseFields


windSpeed_600_nc[:,:,:] = Pred_coarseRes[0,:,:,:]
windDir_600_nc[:,:,:] = Pred_coarseRes[1,:,:,:]
windSpeed_700_nc[:,:,:] = Pred_coarseRes[2,:,:,:]
windDir_700_nc[:,:,:] = Pred_coarseRes[3,:,:,:]
CAPE_nc[:,:,:] = Pred_coarseRes[4,:,:,:]
WindShearSpeed_nc[:,:,:] = Pred_coarseRes[5,:,:,:]
LI_nc[:,:,:] = Pred_coarseRes[6,:,:,:]

## Attributes of the NetCDF:
nc.stream = "s4" # ????
nc.title = 'Sample fine and coarse scale fields over California'
nc.Conventions = "CF-1.0"  # ????
nc.history = "Created ~Mar 2021 by Joseph Bellier" 
nc.institution = "NOAA/ESRL Physical Sciences Laboratory"
nc.platform = "Model" 
nc.references = "None" 

nc.close()